In [ ]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
#m=1.0 #masa
#kB=1.38064852e-23 #cte Boltzmann m^2 kg s^-2 K^-1
#T=300 #temperatura
#sigma=np.sqrt(kB*T/m) 
## Adimensionalizamos el problema
sigma=1
lado=2.0
altura=1.0
def velocidad(sigma): #programa para v perpendicular
    z=rnd.random()
    v=np.sqrt(- 2*sigma**2 *np.log(1-z))
    return v

def cajasin(x,y,vx,vy): #programa para calcular la evolución de la particula sin pistón
    lado=2.0
    altura=1.0
    tarr= (altura-y)/vy #pared superior
    taba=-y/vy #pared inferior
    tizq=-x/vx #pared izquierda
    tder=(lado-x)/vx #pared derecha
    tiempos=[tarr,taba,tizq,tder]
    q=[]
    for i in tiempos:
        if i>0.0: #escogemos aquellos tiempos que sean físicos
            q.append(i)
    t=min(q)
    # Event.driven
    if t==tarr:
        x,y,vx,vy=arriba(x,y,vx,vy,t,sigma)
    elif t==taba:
        x,y,vx,vy=abajo(x,y,vx,vy,t,sigma)
    elif t==tizq:
        x,y,vx,vy=izquierda(x,y,vx,vy,t,sigma)
    elif t==tder:
        x,y,vx,vy=derecha(x,y,vx,vy,t,sigma)
    return x,y,vx,vy,t

def cajapiston(x,y,vx,vy,W,xpiston,vpiston): #programa para calcular los tiempos al meter el pistón
    lado=2.0
    altura=1.0
    tarr= (altura-y)/vy #pared superior
    taba=-y/vy #pared inferior
    tizq=-x/vx #pared izquierda
    tder=(lado-x)/vx #pared derecha
    tpiston=(xpiston-x)/vx
    tiempos=[tarr,taba,tizq,tder,tpiston]
    q=[]
    for i in tiempos:
        if i>0.0:
            q.append(i) # nos quitamos los tiempos negativos y el de la pared
    t=min(q) #escogemos el menor tiempo
    # Event.driven
    if t==tarr:
        x,y,vx,vy=arriba(x,y,vx,vy,t,sigma)
    elif t==taba:
        x,y,vx,vy=abajo(x,y,vx,vy,t,sigma)
    elif t==tizq:
        x,y,vx,vy=izquierda(x,y,vx,vy,t,sigma)
    elif t==tder:
        x,y,vx,vy=derecha(x,y,vx,vy,t,sigma)
    elif t==tpiston:
        x,y,vx,W=piston(x,y,vx,vy,t,W,xpiston,vpiston)
    return x,y,vx,vy,t,W

def arriba(x,y,vx,vy,t,sigma) :
    vpar=rnd.gauss(0,sigma)  
    vper=velocidad(sigma)
    x+=vx*t 
    y=altura
    vx=vpar
    vy=-vper
    return x,y,vx,vy

def abajo(x,y,vx,vy,t,sigma) :
    vpar=rnd.gauss(0,sigma)  
    vper=velocidad(sigma)
    x+=vx*t
    y=0
    vx=vpar
    vy=vper
    return x,y,vx,vy   

def izquierda(x,y,vx,vy,t,sigma) :
    vpar=rnd.gauss(0,sigma)  
    vper=velocidad(sigma)
    x=0
    y+=vy*t
    vx=vper
    vy=vpar
    return x,y,vx,vy   

def derecha(x,y,vx,vy,t,sigma) :
    vpar=rnd.gauss(0,sigma)  
    vper=velocidad(sigma)
    x=lado
    y+=vy*t
    vx=-vper
    vy=vpar
    return x,y,vx,vy  

def piston(x,y,vx,vy,t,W,xpiston,vpiston):
    x=xpiston
    y+=vy*t
    vel=vx
    vx= 2*vpiston -vx
    W -= 2*vpiston*(vel-vpiston)
    return x,y,vx,W
    
def szilard(t1,sigma,vpist):
    lado=2.0
    altura=1.0
    
    x0=rnd.uniform(0.0,lado)
    y0=rnd.uniform(0.0,altura)
    v=velocidad(sigma)
    theta=rnd.random()*2*np.pi
    vx,vy=v*np.cos(theta),v*np.sin(theta)
    xpuntos,ypuntos=[],[]
    tpuntos=[]
    x,y=x0,y0

    tmax=t1
    tiempototal=0.0
    colisiones=0
    distribucion_velocidades=[]
    ## Bloque 1
    while tiempototal<tmax: # Bucle principal
        x,y,vx,vy,t=cajasin(x,y,vx,vy) #calculamos el tiempo a la siguiente colision para avanzarlo "event-driven"
        tiempototal+=t #avanzamos el tiempo de la simulación
        xpuntos.append(x)
        ypuntos.append(y)
        tpuntos.append(tiempototal)
        colisiones+=1
        speed=np.sqrt(vx**2 + vy**2)
        distribucion_velocidades.append(speed)
    print('número de colisiones', colisiones)
    plt.plot(x0,y0,'r*')
    plt.plot(xpuntos,ypuntos)
    plt.plot(xpuntos,ypuntos,'k,') #pintamos los puntos en los que colisiona con la superficie
    plt.xlabel('x')
    plt.ylabel('y')

    plt.show()
    plt.hist(distribucion_velocidades,100)
    plt.show()
    ## Inserción del pistón
    xpiston= lado/2
    ## Medida
    if x<lado/2:
        vpiston= vpist
    elif x>lado/2:  
        vpiston= -vpist
    tiempo =abs(xpiston/vpiston)
    tiempototal=0
    t=0
    W=0
    xpuntos,ypuntos,tpuntos=[],[],[]
    trabajo=[]
    distribucion_velocidades=[]
    while tiempototal<tiempo:
        x,y,vx,vy,t,W=cajapiston(x,y,vx,vy,W,xpiston,vpiston)  #evolución con pistón 
        tiempototal+=t #avanzamos el tiempo de la simulación
        xpiston+=vpiston*t
        if xpiston>lado:
            #print('te has pasado')
            break
        elif xpiston<0:
            #print('te has pasao')
            break
        xpuntos.append(x)
        ypuntos.append(y)
        tpuntos.append(tiempototal)
        trabajo.append(W)
        colisiones+=1
        speed=np.sqrt(vx**2 + vy**2)
        distribucion_velocidades.append(speed)
    print('número de colisiones', colisiones)
    print('W=',W)
    print('-log(2)',-np.log(2))
    plt.plot(x0,y0,'r*')
    plt.plot(xpuntos,ypuntos,'k,') #pintamos los puntos en los que colisiona con la superficie
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()
    plt.hist(distribucion_velocidades,100)
    plt.show()
    plt.plot(tpuntos,trabajo)
    f=[]
    for i in tpuntos:
        f.append(-np.log(2))
    plt.plot(tpuntos,f,'g')
    plt.show()
    np.savetxt('velocidades.dat',distribucion_velocidades)
    return W
